### 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

아래 코드의 주석을 풀고 발급받은 API KEY 를 설정합니다.


In [1]:
import os

# TAVILY API KEY를 기입합니다.
# os.environ["TAVILY_API_KEY"] = "TAVILY API KEY 입력"

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=3)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행합니다.

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행합니다.


In [4]:
# 검색 결과를 가져옵니다.
search.invoke("판교 몽중헌 전화번호는 무엇인가요?")

[{'url': 'http://www.mongjungheon.co.kr/pangyo.jsp',
  'content': '남기는 백주\n500ml 75,000\n농향형 34% 연태지방의 수수를 원료로 만든 향이 좋고 부드러운 백주\n500ml 65,000\n250ml 38,000\n125ml 28,000\nCHAMPAGNE · SPARKLING WINE\n350,000\n250,000\n160,000\n90,000\n70,000\nWHITE WINE\n170,000\n110,000\n95,000\n90,000\n85,000\n75,000\nRED WINE\n380,000\n180,000\n150,000\n130,000\n120,000\n100,000\n95,000\n90,000\n80,000\n75,000\n70,000\nGLASS WINE\n15,000\n15,000\nDISTILLED LIQUOR\n375ml 35,000\n375ml 35,000\nBEER\n320ml 11,000\n640ml 14,000\n580ml 11,000\n600ml 12,000\n610ml 12,000\n330ml 8,000\n330ml 8,000\nSOFT DRINK\n7,000\n7,000\n5,000\n5,000\n4,500\n8,000\nCOFFEE\nhot 5,000\nIced 6,000\nhot 6,000\nIced 7,000\nhot 5,000\nDim Sum Special Course\nLUNCH - 70,000 (1人)\nDINNER - 90,000 (1人)\n Chef-Recommended\nSpicy Dish\n대만\n금문향형 58% 최상의 수질과 기후를 가진 금문도에서 자란 수수를 엄선하여 제조한 술로, 부드럽고 감미로운 맛이 우수한 백주\n600ml 180,000\n300ml 100,000\n貴州省\n귀주성\n장향형\n53%\n수수를 원료로 하여 8번의 발효와 9번의 증류를 거친 후 3년 이상 숙성시킨 술로, 자연 그대로의 발효공법을 통해 오직 좋은 원재료 

### Agent 가 사용할 도구 목록 정의

이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.


In [5]:
# tools 리스트에 search를 추가합니다.
tools = [search]

## Agent with smaller model


In [7]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools.render import render_text_description_and_args
from langchain_core.messages import AIMessage, HumanMessage

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool


local_llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
    temperature=0,
)


@tool
def search_phonenumber(query: str) -> str:
    """장소에 대한 전화번호 검색 결과를 반환할 때 사용되는 도구입니다"""
    return "판교 몽중헌 전화번호: 010-1234-5678\n\n서울 OOO 전화번호: 02-123-4567"


tools = [search_phonenumber]

In [8]:
from typing import Tuple, List

chat_model_with_stop = local_llm.bind(
    stop=["Observation", "\nObservation", "\n관측"])

# Inspiration taken from hub.pull("hwchase17/react-json")
system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action... 
(this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            system_message,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [9]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"]),
        "chat_history": lambda x: (
            _format_chat_history(x["chat_history"]) if x.get(
                "chat_history") else []
        ),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput
)

In [10]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = agent_executor.invoke(
    {
        "input": "판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.",
        "chat_history": [],
    }
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...
<|im_start|>system
Perform the task to the best of your ability.<|im_end|>
<|im_start|>user
Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

search_phonenumber: search_phonenumber(query: str) -> str - 장소에 대한 전화번호 검색 결과를 반환할 때 사용되는 도구입니다, args: {'query': {'title': 'Query', 'type': 'string'}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: ['search_phonenumber']
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}
```
The $JSON_BLOB must always be enclosed with triple backticks!


In [11]:
parser = ReActJsonSingleInputOutputParser()
action = parser.parse(response["output"])

In [12]:
action

AgentAction(tool='search_phonenumber', tool_input={'query': '판교 몽중헌'}, log='the final answer to the original input question\n\nBegin! Reminder to always use the exact characters `Final Answer` when responding.\'\n<|im_end|>\n<|im_start|>user\n판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.<|im_end|>\n<|im_start|>assistant\nQuestion: 판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.\nThought: 사용자로부터 요청을 받았습니다. пангджо мунджухен의 전화를 찾기 위해 \'search_phonenumber\' 도구를 호출할 것입니다.\nAction:```{\n     "action": "search_phonenumber",\n     "action_input": {\n         "query": "판교 몽중헌"\n     }\n}```')

In [13]:
action.tool

'search_phonenumber'

In [14]:
action.tool_input

{'query': '판교 몽중헌'}

In [15]:
print(action.log)

the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
<|im_end|>
<|im_start|>user
판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.<|im_end|>
<|im_start|>assistant
Question: 판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.
Thought: 사용자로부터 요청을 받았습니다. пангджо мунджухен의 전화를 찾기 위해 'search_phonenumber' 도구를 호출할 것입니다.
Action:```{
     "action": "search_phonenumber",
     "action_input": {
         "query": "판교 몽중헌"
     }
}```


In [16]:
from langgraph.prebuilt import ToolExecutor

# ToolExecutor 클래스의 인스턴스를 생성
tool_executor = ToolExecutor(tools)

In [17]:
# 결과 확인
tool_executor.invoke(action)

'판교 몽중헌 전화번호: 010-1234-5678\n\n서울 OOO 전화번호: 02-123-4567'

## 검색 결과를 바탕으로 RAG


In [18]:
from langchain_core.agents import AgentAction
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [19]:
eeve = ChatOllama(
    model="EEVE-Korean-10.8B:long",
    temperature=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [20]:
# 결과 확인
dummy_tool_executor = ToolExecutor([search_phonenumber])
dummy_result = dummy_tool_executor.invoke(action)
dummy_result

'판교 몽중헌 전화번호: 010-1234-5678\n\n서울 OOO 전화번호: 02-123-4567'

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI Assistant. You must answer the question based on the context.\n#Context: {context}",
        ),
        ("user", "{question}"),
    ]
)

chain = prompt | eeve

In [22]:
_ = chain.invoke(
    {
        "question": "판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.",
        "context": dummy_result,
    }
)

판교 몽중헌의 정확한 전화번호는 다음과 같습니다: 010-1234-5678

In [23]:
search_action = AgentAction(
    log="test",
    tool=TavilySearchResults().name,
    tool_input={"query": "판교 몽중헌 전화번호"},
)

In [24]:
tool_executor = ToolExecutor([TavilySearchResults(k=3)])
result = tool_executor.invoke(search_action)
result

[{'url': 'http://www.mongjungheon.co.kr/pangyo.jsp',
  'content': '남기는 백주\n500ml 75,000\n농향형 34% 연태지방의 수수를 원료로 만든 향이 좋고 부드러운 백주\n500ml 65,000\n250ml 38,000\n125ml 28,000\nCHAMPAGNE · SPARKLING WINE\n350,000\n250,000\n160,000\n90,000\n70,000\nWHITE WINE\n170,000\n110,000\n95,000\n90,000\n85,000\n75,000\nRED WINE\n380,000\n180,000\n150,000\n130,000\n120,000\n100,000\n95,000\n90,000\n80,000\n75,000\n70,000\nGLASS WINE\n15,000\n15,000\nDISTILLED LIQUOR\n375ml 35,000\n375ml 35,000\nBEER\n320ml 11,000\n640ml 14,000\n580ml 11,000\n600ml 12,000\n610ml 12,000\n330ml 8,000\n330ml 8,000\nSOFT DRINK\n7,000\n7,000\n5,000\n5,000\n4,500\n8,000\nCOFFEE\nhot 5,000\nIced 6,000\nhot 6,000\nIced 7,000\nhot 5,000\nDim Sum Special Course\nLUNCH - 70,000 (1人)\nDINNER - 90,000 (1人)\n Chef-Recommended\nSpicy Dish\n대만\n금문향형 58% 최상의 수질과 기후를 가진 금문도에서 자란 수수를 엄선하여 제조한 술로, 부드럽고 감미로운 맛이 우수한 백주\n600ml 180,000\n300ml 100,000\n貴州省\n귀주성\n장향형\n53%\n수수를 원료로 하여 8번의 발효와 9번의 증류를 거친 후 3년 이상 숙성시킨 술로, 자연 그대로의 발효공법을 통해 오직 좋은 원재료 

In [25]:
search_result = " ".join([r["content"] for r in result])
search_result

"남기는 백주\n500ml 75,000\n농향형 34% 연태지방의 수수를 원료로 만든 향이 좋고 부드러운 백주\n500ml 65,000\n250ml 38,000\n125ml 28,000\nCHAMPAGNE · SPARKLING WINE\n350,000\n250,000\n160,000\n90,000\n70,000\nWHITE WINE\n170,000\n110,000\n95,000\n90,000\n85,000\n75,000\nRED WINE\n380,000\n180,000\n150,000\n130,000\n120,000\n100,000\n95,000\n90,000\n80,000\n75,000\n70,000\nGLASS WINE\n15,000\n15,000\nDISTILLED LIQUOR\n375ml 35,000\n375ml 35,000\nBEER\n320ml 11,000\n640ml 14,000\n580ml 11,000\n600ml 12,000\n610ml 12,000\n330ml 8,000\n330ml 8,000\nSOFT DRINK\n7,000\n7,000\n5,000\n5,000\n4,500\n8,000\nCOFFEE\nhot 5,000\nIced 6,000\nhot 6,000\nIced 7,000\nhot 5,000\nDim Sum Special Course\nLUNCH - 70,000 (1人)\nDINNER - 90,000 (1人)\n Chef-Recommended\nSpicy Dish\n대만\n금문향형 58% 최상의 수질과 기후를 가진 금문도에서 자란 수수를 엄선하여 제조한 술로, 부드럽고 감미로운 맛이 우수한 백주\n600ml 180,000\n300ml 100,000\n貴州省\n귀주성\n장향형\n53%\n수수를 원료로 하여 8번의 발효와 9번의 증류를 거친 후 3년 이상 숙성시킨 술로, 자연 그대로의 발효공법을 통해 오직 좋은 원재료 만으로 빚어낸 중국의 대표 명주\n500ml 1,500,000\n농향형\n33%\n중국 명주를 만드는 강 ‘적수하’의 

In [26]:
_ = chain.invoke(
    {
        "question": "판교 몽중헌 전화번호를 웹 검색하여 결과를 알려주세요.",
        "context": search_result,
    }
)

판교 몽중헌의 전화번호는 다음과 같습니다: 031-601-7574.